<a href="https://colab.research.google.com/github/Vishal-ib97/Developer-Name-Disambiguation/blob/main/data_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from IPython.core.display import HTML
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from bs4 import BeautifulSoup
import datetime
import os
import re
import pandas as pd
import numpy as np
import sys
import re
import unicodedata

In [ ]:
#mount drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
new_df = pd.read_csv("/content/gdrive/MyDrive/GG_Project_NLP/Data/commit_messages/new_commits_df.csv")
com =  pd.read_csv("/content/gdrive/MyDrive/GG_Project_NLP/Data/commit_messages/json_data_updated.csv")

In [ ]:
#filter projects based on the projects present in the ground truth page
new_df = new_df[new_df.proj_name.isin(com.project.values)]
new_df.author_name = new_df.author_name.str.lower()
new_df.drop_duplicates(subset = ['author_name', 'email_id'], inplace = True)
new_df.drop(['Unnamed: 0'], axis = 1, inplace = True)

#dealing with Nans in author name
#checking if the corresponding email ids contain any useful information and filling the rows
new_df.loc[new_df.email_id == "<!-- Author: Scott Boag -->", "author_name"] = "Scott Boag"
new_df.loc[new_df.email_id == "<!-- Author: Scott Boag -->", "email_id"] = "sboag"
new_df.loc[new_df.email_id == "<!-- Template for SearchWidget. Author: hearnden@google.com (David Hearnden) -->", "author_name"] = "David Hearnden"
new_df.loc[new_df.email_id == "<!-- Template for SearchWidget. Author: hearnden@google.com (David Hearnden) -->", "email_id"] = "hearnden"
new_df.loc[new_df.email_id == "<!-- Author: Jordan Naftolin -->", "author_name"] = "Jordan Naftolin"
new_df.loc[new_df.email_id == "<!-- Author: Jordan Naftolin -->", "email_id"] = "jordan"
new_df.loc[new_df.email_id == "<!-- Author: jeremias.maerki@outline.ch -->", "email_id"] = "jeremias.maerki"
new_df.loc[new_df.email_id == "<!-- Author: jeremias.maerki@outline.ch -->", "author_name"] = "Jeremias Maerki"
new_df.loc[new_df.email_id == "<!-- Author: Paul Dick -->", "author_name"] = "Paul Dick"
new_df.loc[new_df.email_id == "<!-- Author: Paul Dick -->", "email_id"] = "pauldick"
new_df.loc[new_df.email_id == "<!-- Author: shane_curcuru@us.ibm.com -->", "author_name"] = "Shane Curcuru"
new_df.loc[new_df.email_id == "<!-- Author: shane_curcuru@us.ibm.com -->", "email_id"] = "shane_curcuru"
new_df.to_csv("/content/gdrive/MyDrive/GG_Project_NLP/Data/commit_messages/commit_10_proj_df.csv")

In [ ]:
g = new_df.groupby('proj_name')
proj_commit = pd.DataFrame()
new_df.head()

,proj_name,date_time,author_name,email_id
0,accumulo,2020-11-02 15:14:00,mike miller,<mmiller@apache.org>
1,accumulo,2020-11-03 00:53:00,christopher tubbs,<ctubbsii@apache.org>
2,accumulo,2020-11-03 17:39:00,keith turner,<kturner@apache.org>
3,accumulo,2020-11-04 18:25:00,jeffrey manno,<jeffreymanno15@gmail.com>
4,accumulo,2020-11-04 18:32:00,buildbot,<users@infra.apache.org>


In [ ]:
#processing the commit data separately for each project since projects ahve shared contibutors

for pr in g.groups:
  print(pr)
  new_df = pd.read_csv("/content/gdrive/MyDrive/GG_Project_NLP/Data/commit_messages/commit_10_proj_df.csv")
  new_df.drop(['Unnamed: 0'], inplace = True, axis = 1)

  new_df = new_df[new_df.proj_name == pr]
  new_df.reset_index(inplace = True)
  np.set_printoptions(threshold=sys.maxsize)
  pd.set_option('display.max_columns', None)

  com.author_name = com.author_name.str.lower()
  new_df.author_name = new_df.author_name.str.lower()
  new_df.author_name = new_df.author_name.astype('str')
  #new_df.email_id = new_df.email_id.astype('str')
  new_df = new_df[~new_df.author_name.str.contains(",")]

  new_df.author_name = new_df.author_name.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
  new_df = new_df[new_df.author_name != ""]

  def clean_auth_names():
    df = new_df
    df.loc[df.email_id == "barboni", "author_name"] = "eric barboni"
    df.drop(df[df['author_name'].isnull()].index, inplace=True)
    df['author_name'] = df['author_name'].str.lower()
    replace_str = [ ",", "\):", "/", "\\", "*", "$", "`", "\'", "\"", "|", ",", " \)", "=", "+", "#", "=", "\[", "\]", "?", "\(\)\;", "\);", ";"]
    for i in replace_str:
      df['author_name'] = df['author_name'].str.replace(i,"")

    df['author_name'] = df['author_name'].str.replace("  "," ")
    bot_str = ['\[bot\]', 'buildbot', 'nicerobot', 'github bot', '-bot', 'github actions bot', 'mergebot', 'renovate bot', 'automated github actions', 'auth of last commit', '4 immediate children', 'utf-8', 'entry get author', 'gg@localhost', 'dependabot[bot]']
    for i in bot_str:
      df = df.drop(df[df['author_name'].str.contains(i)].index)
    

    df = new_df.drop(df[df['author_name']=='root'].index)
  #  df['author_name'] = df['author_name'].str.strip()
    df['author_name'] = df['author_name'].str.strip()
    return df
  new_df = clean_auth_names()


  def cleanhtml(raw_html):
    if "@" not in raw_html:
      cleanr = re.compile('<.*?>')
      new_df.loc[new_df.email_id == raw_html,'email_id'] = re.sub(cleanr, '', raw_html)

  def clean_emailids(new_df):
    new_df['email_id'] = new_df['email_id'].str.lower()
    new_df.drop(new_df[new_df['email_id'].str.contains("noreply", na = False)].index, inplace = True)
    #df = df.drop(df[df['email_id'].str.contains("\*\*", na=False)].index)
    
    new_df['email_id'].apply(lambda x: cleanhtml(str(x)))
    new_df['email_id'] = new_df['email_id'].str.split('@').str[0]
    new_df['email_id'] = new_df['email_id'].str.replace("<", "")

    new_df.drop(new_df[new_df['email_id']=="root"].index, inplace = True)
    new_df.drop(new_df[new_df['email_id']=="franta-git"].index, inplace = True)
    new_df.drop(new_df[new_df['email_id']=="count.apache.org"].index, inplace = True)
    new_df.drop(new_df[new_df['email_id']=="do-not-reply"].index, inplace = True)
    new_df.drop(new_df[new_df['email_id']=="github.coffee"].index, inplace = True)
  
    new_df.drop(new_df[new_df['email_id']=="dev-null"].index, inplace = True)
    new_df.drop(new_df[new_df['email_id'].str.contains("-github", na = False)].index, inplace = True)
    new_df.drop(new_df[new_df['email_id'].str.contains("\+github", na = False)].index, inplace = True)
    new_df.drop(new_df[new_df['email_id'].str.contains("\+git", na = False)].index, inplace = True)
    new_df.drop(new_df[new_df['email_id'].str.contains("-git", na = False)].index, inplace = True)
    new_df.drop(new_df[new_df['email_id'].str.contains("github", na = False)].index, inplace = True)
    new_df.drop(new_df[new_df['email_id'].str.contains("git", na = False)].index, inplace = True)

    return new_df
  new_df = clean_emailids(new_df)

  com.author_name = com.author_name.str.lower()
  new_df.author_name = new_df.author_name.str.lower()
  new_df.author_name = new_df.author_name.astype('str')
  new_df.email_id = new_df.email_id.astype('str')
  #comma separated useless entries
  new_df = new_df[~new_df.author_name.apply(lambda x: len(x.split(",")) > 2)]

  #removing author names with two entries because they don't exist in the exhaustive list of committers
  new_df = new_df[~new_df.author_name.apply(lambda x: "@" in x and len(x.split(",")) >= 2)]

  #handling names and email_ids in brackets
  def info_from_brac(x):
    ind = re.search(r'\(.*\)', str(x)).span()
    a = x[ind[0] + 1: ind[1]-1]
    b = x[:ind[0]]
    if "@" in a:
      s = a.split("@")
      if "" not in s and len(s) > 1 and new_df.loc[new_df.author_name == x, "email_id"].isna().any():
        id = s[0]
        new_df.loc[new_df.author_name == x, "email_id"] = " ".join(id.split())
      new_df.loc[new_df.author_name == x, "author_name"] = " ".join(b.split())
    elif "@" in b:
      s = b.split("@")
      if "" not in s and len(s) > 1 and new_df.loc[new_df.author_name == x, "email_id"].isna().any():
        id = s[0]
        new_df.loc[new_df.author_name == x, "email_id"] = " ".join(id.split())
      new_df.loc[new_df.author_name == x, "author_name"] = " ".join(a.split())

  new_df[(new_df.author_name.str.contains("@"))&(new_df.author_name.str.contains("\("))].author_name.apply(lambda x: info_from_brac(x));

  #removing unwanted patterns
  new_df['author_name'] = new_df['author_name'].str.lower()
  #replace_str = ["&gt;", "&lt;", ",", "email:", "\):", " \)", "=", "+", "#", "=", "\[", "\]", "?", "\(\)\;", "\);", ";"]
  replace_str = ["&gt;", "&lt;", ",", "email:", "\):", "/", "\\", "*", "$", "`", "\'", "\"", "|", " \)", "=", "+", "#", "=", "\[", "\]", "?", "\(\)\;", "\);", ";"]
  for i in replace_str:
    new_df['author_name'] = new_df['author_name'].str.replace(i,"")
    com['author_name'] = com['author_name'].str.replace(i,"")

  new_df['author_name'] = new_df['author_name'].str.replace("  "," ")
  bot_str = ['\[bot\]', 'buildbot', 'nicerobot', 'github bot', '-bot', 'github actions bot', 'mergebot', 'renovate bot', 'automated github actions', 'auth of last commit', '4 immediate children', 'utf-8', 'entry get author', 'gg@localhost']
  for i in bot_str:
    new_df = new_df.drop(new_df[new_df['author_name'].str.contains(i)].index)
  new_df = new_df.drop(new_df[new_df['author_name']=='root'].index)

  new_df['author_name'] = new_df['author_name'].apply(lambda x: " ".join(x.split()))


  #moving user_ids from author_name to email_id
  def replace_id_in_author(x):
    for i in new_df.loc[new_df.author_name == x].index:
      if new_df.loc[new_df.index == i, "email_id"].isna().any(): 
        new_df.loc[new_df.index == i, "email_id"] = x
        new_df.loc[new_df.index == i, "author_name"] = com[com.user_ids == x].author_name.values[0].lower()

  new_df[new_df.author_name.isin(com.user_ids.values)].author_name.apply(lambda x: replace_id_in_author(x))

  #same as above but for non-NaNs in email_id
  new_rows = []
  def get_dict_for_rows(x):
    for i in new_df.loc[new_df.author_name == x].index:
      dit = new_df[new_df.index == i].to_dict(orient = 'records')[0]
      dit["email_id"] = x
      dit["author_name"] = com[com.user_ids == x].author_name.values[0]
      new_df.loc[new_df.index == i, "author_name"] = com[com.user_ids == x].author_name.values[0].lower()
      new_rows.append(dit)

  new_df[new_df.author_name.isin(com.user_ids.values)].author_name.apply(lambda x: get_dict_for_rows(x));
  new_df = pd.concat([new_df, pd.DataFrame(new_rows)])

  #handling entries with "@" in author_name
  rem = ["@gregfranko", "@pah", "jason moon - @jsonmoon", "cloudbees dev@cloud", "smashew @ 2013-08-26", 
        "rich @ radics", "tim urberg - tim_urberg@yahoo.com", "_ liuyu66@", "trevor a.k.a @rawkintrevo", "cedric.pemberton@gmail.colm",
        "steve blackmon @steveblackmon", "nayananga@acerubuntu18.04", "idevai", "clebert suconic at redhat dot com"]
  new_df = new_df[~new_df.author_name.isin(rem)]

  new_df.loc[new_df.author_name == "jon haddad jon@jonhaddad.com", "email_id"] = "jon"
  new_df.loc[new_df.author_name == "jon haddad jon@jonhaddad.com", "author_name"] = "jon haddad"
  new_df.loc[new_df.author_name == "andrzej bialecki ab@getopt.org", "email_id"] = "ab"
  new_df.loc[new_df.author_name == "andrzej bialecki ab@getopt.org", "author_name"] = "andrzej bialecki"
  new_df.loc[new_df.author_name == "marc prud'hommeaux mprudhom@apache.org", "email_id"] = "mprudhom"
  new_df.loc[new_df.author_name == "marc prud'hommeaux mprudhom@apache.org", "author_name"] = "marc prud'hommeaux"
  new_df.loc[new_df.author_name == "julia wang jwang98052@yahoo.com", "email_id"] = "jwang98052"
  new_df.loc[new_df.author_name == "julia wang jwang98052@yahoo.com", "author_name"] = "julia wang"
  new_df.loc[new_df.author_name == "markus weimer weimer@apache.org", "email_id"] = "weimer"
  new_df.loc[new_df.author_name == "markus weimer weimer@apache.org", "author_name"] = "markus weimer"
  new_df.loc[new_df.author_name == "henri yandell bayard@generationjava.com", "email_id"] = "bayard"
  new_df.loc[new_df.author_name == "henri yandell bayard@generationjava.com", "author_name"] = "henri yandell"

  if pr == "cassandra":
    more_rows = [{"proj_name" : "cassandra", "date_time" : "2020-04-01 15:16:00","author_name" : "jon haddad", "email_id" : "rustyrazorblade"}]
    new_df = pd.concat([new_df, pd.DataFrame(more_rows)])
  if pr == "reef":
    more_rows = [{"proj_name" : "reef", "date_time" : "2015-07-15 16:59:00","author_name" : "julia wang", "email_id" : "juliaw"}]
    new_df = pd.concat([new_df, pd.DataFrame(more_rows)])

  def add_id_to_email(x):
    sp_temp = x
    if len(x.split(maxsplit = 1)) > 1:
      sp_temp = x.split()[1]
      sp_temp = " ".join(sp_temp.split())
    sp = x.split("@")[0]
    sp = " ".join(sp.split())
    for i in new_df.loc[new_df.author_name == x].index:
      if new_df.loc[new_df.index == i, "email_id"].isna().any():
        new_df.loc[new_df.index == i, "email_id"] = sp
        if sp in com.user_ids.values:
          new_df.loc[new_df.index == i, "author_name"] = com.loc[com.user_ids == sp, "author_name"].values[0].lower()

  new_df[new_df.author_name.str.contains("@")].author_name.apply(lambda x: add_id_to_email(x));

  new_rows_2 = []
  def get_dict_for_rows_2(x):
    sp_temp = x
    if len(x.split(maxsplit = 1)) > 1:
      sp_temp = x.split()[1]
      sp_temp = " ".join(sp_temp.split())
    sp = x.split("@")[0]
    sp = " ".join(sp.split())
    for i in new_df.loc[new_df.author_name == x].index:
      a = new_df.loc[new_df.index == i, "email_id"].values[0].split("@")[0]
      a = " ".join(a.split())
      if a == sp:
        if sp in com.user_ids.values:
          new_df.loc[new_df.index == i, "author_name"] = com.loc[com.user_ids == sp, "author_name"].values[0].lower()
        else:
          new_df.loc[new_df.index == i, "author_name"] = sp
      else:
        dit = new_df[new_df.index == i].to_dict(orient = 'records')[0]
        
        dit["email_id"] = sp
        if sp in com.user_ids.values:
          dit["author_name"] = com[com.user_ids == sp].author_name.values[0]
          new_df.loc[new_df.index == i, "author_name"] = com[com.user_ids == sp].author_name.values[0].lower()
        elif a in com.user_ids.values:
          dit["author_name"] = com[com.user_ids == a].author_name.values[0]
          new_df.loc[new_df.index == i, "author_name"] = com[com.user_ids == a].author_name.values[0].lower()
        else:
          dit["author_name"] = sp
          new_df.loc[new_df.index == i, "author_name"] = a
        new_rows_2.append(dit)

  new_df[new_df.author_name.str.contains("@")].author_name.apply(lambda x: get_dict_for_rows_2(x));
  new_df = pd.concat([new_df, pd.DataFrame(new_rows_2)])

  new_df.loc[new_df.author_name == "thierry kormann", "email_id"] = "tkormann"
  new_df.loc[new_df.author_name == "anirban roy", "email_id"] = "anirbanroy"

  new_df.reset_index(inplace = True)

  #filling NAs in email id using the json file data if the author name is present in the json data file
  def fill_na_email(x):
    for i in new_df.loc[new_df.author_name == x].index:
      if new_df.loc[new_df.index == i, "email_id"].isna().any() and x in com.author_name.values:
        new_df.loc[new_df.index == i, "email_id"] = com[com.author_name == x].user_ids.values[0].lower()

  new_df[new_df.email_id.isna()].author_name.apply(lambda x: fill_na_email(x))

  new_df.loc[new_df.email_id.apply(lambda x: len(x) == 1), "email_id"] = new_df.loc[new_df.email_id.apply(lambda x: len(x) == 1), "author_name"].apply(lambda x : com[com.author_name == x].user_ids.values[0].lower() if x in com.author_name.values else np.nan)
  
  new_df.loc[new_df.email_id == "me", "email_id"] = new_df.loc[new_df.email_id == "me", "author_name"].apply(lambda x : com[com.author_name == x].user_ids.values[0].lower() if x in com.author_name.values else np.nan)

  new_df = new_df[new_df.email_id != "dev"]
  new_df = new_df[~new_df.author_name.str.isnumeric()]
  new_df = new_df[new_df.author_name != ""]
  new_df = new_df[new_df.author_name != "nan"]

  replace_str = ["&gt;", "&lt;", ",", "email:", "\):", "/", "\\", "*", "$", "`", "\'", "\"", "|", " \)", "=", "+", "#", "=", "\[", "\]", "?", "\(\)\;", "\);", ";", ">", "<"]
  for i in replace_str:
    new_df['author_name'] = new_df['author_name'].str.replace(i,"")
    #new_df['email_id'] = new_df['email_id'].str.replace(i,"")

  new_df = new_df[new_df.author_name != ""]
  new_df = new_df[new_df.email_id != ""]

  proj_commit = pd.concat([proj_commit, new_df])

proj_commit.author_name = proj_commit.author_name.str.replace("-", "")
proj_commit.to_csv("/content/gdrive/MyDrive/GG_Project_NLP/Data/commit_messages/final_df_curr_10.csv")

In [ ]:
#more cleaning...
#Merge preprocessed and from/sub df
df_tot_commit = pd.read_csv('/content/drive/MyDrive/GG_Project_NLP/Data/commit_messages/final_df_curr_10.csv')
df_from_sub = pd.read_csv('/content/gdrive/MyDrive/GG_Project_NLP/Data/commit_messages/commit_data_from_subject_10.csv')

print("tot_commit_columns: ", df_tot_commit.columns)
print("tot_commit_length: ", len(df_tot_commit))
print("from_subject_columns: ", df_from_sub.columns)
print("from_subject_length: ", len(df_from_sub))

merged_df = pd.merge(df_tot_commit, df_from_sub,  how='left', on = ['proj_name', 'date_time'], validate='many_to_many')
print(df_tot_commit.isna().sum().sort_values(ascending=False))

#drop names with "jira" since they are not part of the committers
merged_df.drop(merged_df[merged_df['from'].str.contains("(jira)", na = False)].index, inplace = True)
merged_df.drop(merged_df[merged_df['from'].str.contains("(JIRA)", na = False)].index, inplace = True)
merged_df.drop(merged_df[merged_df['from'].str.contains("(Jira)", na = False)].index, inplace = True)
merged_df.dropna(subset=['author_name','email_id'], how='all',inplace=True)

#create dataframes with and without email NAs
no_na_email = merged_df[~(merged_df['email_id'].isna())]
na_email=merged_df[merged_df['email_id'].isna()]
#na_email.to_csv('/content/drive/MyDrive/GG_Project_NLP/Data/commit_messages/NA_email_merged_updated_10.csv')

#Manually filled the Nans and stored in the same csv file. Now merge with the df without nans in email
merged_no_na_email = no_na_email.append(na_email, ignore_index=True)
merged_no_na_email.drop_duplicates(subset=['proj_name','author_name', 'email_id', 'from'], inplace=True)
merged_no_na_email.drop(columns=['Unnamed: 0_x','level_0', 'Unnamed: 0.1', 'Unnamed: 0_y', 'from', 'subject', 'commits_folder', 'Unnamed: 0'], inplace=True)


merged_no_na_email.drop(index = [4106, 4405], inplace=True)
#remove brackets and anything that's inside them
merged_no_na_email['author_name'] = merged_no_na_email['author_name'].str.replace(r"\(.*\)","")
merged_no_na_email['author_name'] = merged_no_na_email['author_name'].str.replace("\(", "")
merged_no_na_email[(merged_no_na_email['author_name'].str.contains("\(")) | (merged_no_na_email['author_name'].str.contains("\)"))]

#removing entries with just numbers in the email id column
merged_no_na_email = merged_no_na_email[~merged_no_na_email['email_id'].str.isnumeric()]
#merged_no_na_email.to_csv('/content/drive/MyDrive/GG_Project_NLP/Data/commit_messages/final_cleaned_10.csv')

In [ ]:
#cleaning the hidden email ids in 'from' 
new_df_10 = pd.read_csv("/content/gdrive/MyDrive/GG_Project_NLP/Data/commit_messages/final_cleaned_10.csv")
from_df = new_df_10[['proj_name', 'author_name', 'email_id', 'from']]
from_df.dropna(inplace = True)
from_df = from_df[from_df['from'] != "GitBox"]
#from_df['from'] = from_df['from'].apply(lambda x: x[:x.index(".")] if "." in x else x)
from_df = from_df[from_df['from'] != ""]
from_df = from_df[~from_df['from'].str.contains("Apache Jenkins")]
from_df = from_df[~from_df['from'].str.contains("github")]
from_df['from_ids'] = from_df['from'].apply(lambda x: x[:x.index(".")] if "." in x else x)
from_df.to_csv("/content/gdrive/MyDrive/GG_Project_NLP/Data/commit_messages/final_from_df.csv")
from_df.head()